In [0]:
# Check GPU on Google Colab
# !nvidia-smi

In [2]:
# !pip3 install torch torchvision

In [0]:
#import torch
#torch.__version__
#torch.cuda.is_available()

In [4]:
# !pip install -U gensim

    100% |████████████████████████████████| 24.2MB 1.6MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [1]:
#import nltk
#nltk.download("punkt")

[nltk_data] Downloading package punkt to C:\Users\Yan
[nltk_data]     Zheng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk import word_tokenize, sent_tokenize 

In [4]:
from collections import namedtuple

import numpy as np
from tqdm import tqdm

import pandas as pd

from gensim.corpora import Dictionary

import torch
from torch import nn, optim, tensor, autograd
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")
sns.set(rc={'figure.figsize':(12, 8)})


torch.manual_seed(42)

In [6]:
# Read data
# text = open('pickup_lines_all.txt', 'r', encoding = "utf8").read().splitlines()

In [57]:
import json

In [58]:
data1 = []
with open('pickup_lines_cleaned.json', 'r', encoding = "utf8") as json_data:
    for line in json_data: 
        data1.append(json.loads(line))

data2 = []
for line in range(len(data1)): 
    data2.append(data1[line]["text"])

In [61]:
# Remove duplicated and empty data
text = [i for i in list(set(data2)) if i != '']

In [65]:
text[:5]

['Java number I can call you up at?',
 'I tried to send you something sexy last night but the mail man told me to get out of the mailbox',
 "I think I'm called to marriage, can I call you sometime?",
 "Is that a keg in your pants? Because I'd love to tap that ass.",
 'Your cave or mine?']

In [25]:
len(text)

17528

In [63]:
# Tokenize the text.
# tokenized_text = [[list(map(str.lower, word_tokenize(word))) for word in sent_tokenize(line)] for line in text]
tokenized_text = [list(map(str.lower, word_tokenize(line))) for line in text]

In [39]:
tokenized_text[0]

15

In [64]:
max([len(token) for token in tokenized_text])

171

In [66]:
count = 0
for token in tokenized_text: 
    if len(token) > 1000: 
        count += 1
print(count)    


0


In [67]:
class PickupLines(nn.Module):
    def __init__(self, texts):
        self.texts = texts
        
        # Initialize the vocab 
        special_tokens = {'<pad>': 0, '<unk>':1, '<s>':2, '</s>':3}
        self.vocab = Dictionary(texts)
        self.vocab.patch_with_special_tokens(special_tokens)
        
        # Keep track of the vocab size.
        self.vocab_size = len(self.vocab)
        
        # Keep track of how many data points.
        self._len = len(texts)
        
        # Find the longest text in the data.
        self.max_len = max(len(txt) for txt in texts) 
        
    def __getitem__(self, index):
        vectorized_sent = self.vectorize(self.texts[index])
        x_len = len(vectorized_sent)
        # To pad the sentence:
        # Pad left = 0; Pad right = max_len - len of sent.
        pad_dim = (0, self.max_len - len(vectorized_sent))
        vectorized_sent = F.pad(vectorized_sent, pad_dim, 'constant')
        return {'x':vectorized_sent[:-1], 
                'y':vectorized_sent[1:], 
                'x_len':x_len}
    
    def __len__(self):
        return self._len
    
    def vectorize(self, tokens, start_idx=2, end_idx=3):
        """
        :param tokens: Tokens that should be vectorized. 
        :type tokens: list(str)
        """
        # See https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx 
        # Lets just cast list of indices into torch tensors directly =)
        
        vectorized_sent = [start_idx] + self.vocab.doc2idx(tokens) + [end_idx]
        return torch.tensor(vectorized_sent)
    
    def unvectorize(self, indices):
        """
        :param indices: Converts the indices back to tokens.
        :type tokens: list(int)
        """
        return [self.vocab[i] for i in indices]

In [68]:
pickuplines_data = PickupLines(tokenized_text)

In [69]:
len(pickuplines_data.vocab)

17269

In [70]:
batch_size = 10
dataloader = DataLoader(dataset=pickuplines_data, batch_size=batch_size, shuffle=True)

for data_dict in dataloader:
    # Sort indices of data in batch by lengths.
    sorted_indices = np.array(data_dict['x_len']).argsort()[::-1].tolist()
    data_batch = {name:_tensor[sorted_indices]
                  for name, _tensor in data_dict.items()}
    print(data_batch)
    break

{'x': tensor([[   2,   22, 5505,  ...,    0,    0,    0],
        [   2,    4, 1321,  ...,    0,    0,    0],
        [   2,    4, 6817,  ...,    0,    0,    0],
        ...,
        [   2,   44, 1359,  ...,    0,    0,    0],
        [   2,   57,    8,  ...,    0,    0,    0],
        [   2,  285,  106,  ...,    0,    0,    0]]), 'y': tensor([[   22,  5505,  9890,  ...,     0,     0,     0],
        [    4,  1321,   147,  ...,     0,     0,     0],
        [    4,  6817,    18,  ...,     0,     0,     0],
        ...,
        [   44,  1359,   841,  ...,     0,     0,     0],
        [   57,     8,    73,  ...,     0,     0,     0],
        [  285,   106, 15680,  ...,     0,     0,     0]]), 'x_len': tensor([28, 24, 20, 19, 19, 19, 18, 16, 16, 12])}


In [71]:
class Generator(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super(Generator, self).__init__()

        # Initialize the embedding layer with the 
        # - size of input (i.e. no. of words in input vocab)
        # - no. of hidden nodes in the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        
        # Initialize the GRU with the 
        # - size of the input (i.e. embedding layer)
        # - size of the hidden layer 
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers, batch_first=True)
        
        # Initialize the "classifier" layer to map the RNN outputs
        # to the vocabulary. Remember we need to -1 because the 
        # vectorized sentence we left out one token for both x and y:
        # - size of hidden_size of the GRU output.
        # - size of vocabulary
        self.classifier = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, inputs, use_softmax=False, hidden=None):
        # Look up for the embeddings for the input word indices.
        embedded = self.embedding(inputs)
        # Put the embedded inputs into the GRU.
        output, hidden = self.gru(embedded, hidden)
        
        # Matrix manipulation magic.
        batch_size, sequence_len, hidden_size = output.shape
        # Technically, linear layer takes a 2-D matrix as input, so more manipulation...
        output = output.contiguous().view(batch_size * sequence_len, hidden_size)
        # Apply dropout.
        output = F.dropout(output, 0.5)
        # Put it through the classifier
        # And reshape it to [batch_size x sequence_len x vocab_size]
        output = self.classifier(output).view(batch_size, sequence_len, -1)
        
        return (F.softmax(output,dim=2), hidden) if use_softmax else (output, hidden)
        

In [72]:
# Set the hidden_size of the GRU 
embed_size = 12
hidden_size = 10
num_layers = 1

_encoder = Generator(len(pickuplines_data.vocab), embed_size, hidden_size, num_layers)

In [73]:
# Take a batch.
batch_size = 15
dataloader = DataLoader(dataset=pickuplines_data, batch_size=batch_size, shuffle=True)

_batch = next(iter(dataloader))
_inputs, _lengths = _batch['x'], _batch['x_len']
_targets = _batch['y']

In [74]:
_output, _hidden = _encoder(_inputs)
print('Output sizes:\t', _output.shape)
print('Input sizes:\t', batch_size, pickuplines_data.max_len -1, len(pickuplines_data.vocab))
print('Target sizes:\t', _targets.shape)

Output sizes:	 torch.Size([15, 170, 17269])
Input sizes:	 15 170 17269
Target sizes:	 torch.Size([15, 170])


In [75]:
_output.shape

torch.Size([15, 170, 17269])

In [76]:
_output[-1].shape

torch.Size([170, 17269])

In [77]:
_, predicted_indices = torch.max(_output, dim=1)
print(predicted_indices.shape)

torch.Size([15, 17269])


In [78]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

_hyper = ['embed_size', 'hidden_size', 'num_layers',
          'loss_func', 'learning_rate', 'optimizer', 'batch_size']
Hyperparams = namedtuple('Hyperparams', _hyper)


hyperparams = Hyperparams(embed_size=250, hidden_size=250, num_layers=1,
                          loss_func=nn.CrossEntropyLoss,
                          learning_rate=0.03, optimizer=optim.Adam, batch_size=245)

hyperparams

Hyperparams(embed_size=250, hidden_size=250, num_layers=1, loss_func=<class 'torch.nn.modules.loss.CrossEntropyLoss'>, learning_rate=0.03, optimizer=<class 'torch.optim.adam.Adam'>, batch_size=245)

In [79]:
# Training routine.
def train(num_epochs, dataloader, model, criterion, optimizer):
    losses = []
    plt.ion()
    for _e in range(num_epochs):
        for batch in tqdm(dataloader):
            # Zero gradient.
            optimizer.zero_grad()
            x = batch['x'].to(device)
            x_len = batch['x_len'].to(device)
            y = batch['y'].to(device)
            # Feed forward. 
            output, hidden = model(x, use_softmax=False)
            # Compute loss:
            # Shape of the `output` is [batch_size x sequence_len x vocab_size]
            # Shape of `y` is [batch_size x sequence_len]
            # CrossEntropyLoss expects `output` to be [batch_size x vocab_size x sequence_len]
            _, prediction = torch.max(output, dim=2)
            loss = criterion(output.permute(0, 2, 1), y)
            loss.backward()
            optimizer.step()
            losses.append(loss.float().data)

        clear_output(wait=True)
        plt.plot(losses)
        plt.pause(0.05)


def initialize_data_model_optim_loss(hyperparams):
    # Initialize the dataset and dataloader.
    pickuplines_data = PickupLines(tokenized_text)
    dataloader = DataLoader(dataset=pickuplines_data, 
                            batch_size=hyperparams.batch_size, 
                            shuffle=True)

    # Loss function.
    criterion = hyperparams.loss_func(ignore_index=pickuplines_data.vocab.token2id['<pad>'], 
                                      reduction='mean')

    # Model.
    model = Generator(len(pickuplines_data.vocab), hyperparams.embed_size, 
                      hyperparams.hidden_size, hyperparams.num_layers).to(device)

    # Optimizer.
    optimizer = hyperparams.optimizer(model.parameters(), lr=hyperparams.learning_rate)
    
    return dataloader, model, optimizer, criterion

In [80]:
def generate_example(model, temperature=1.0, max_len=100, hidden_state=None):
    start_token, start_idx = '<s>', 2
    # Start state.
    inputs = torch.tensor(pickuplines_data.vocab.token2id[start_token]).unsqueeze(0).unsqueeze(0).to(device)

    sentence = [start_token]
    i = 0
    while i < max_len and sentence[-1] not in ['</s>', '<pad>']:
        i += 1
        
        embedded = model.embedding(inputs)
        output, hidden_state = model.gru(embedded, hidden_state)

        batch_size, sequence_len, hidden_size = output.shape
        output = output.contiguous().view(batch_size * sequence_len, hidden_size)    
        output = model.classifier(output).view(batch_size, sequence_len, -1).squeeze(0)
        #_, prediction = torch.max(F.softmax(output, dim=2), dim=2)
        
        word_weights = output.div(temperature).exp().cpu()
        if len(word_weights.shape) > 1:
            word_weights = word_weights[-1] # Pick the last word.    
        word_idx = torch.multinomial(word_weights, 1).view(-1)
        
        sentence.append(kilgariff_data.vocab[int(word_idx)])
        
        inputs = tensor([pickuplines_data.vocab.token2id[word] for word in sentence]).unsqueeze(0).to(device)
    print(' '.join(sentence))

In [ ]:
hyperparams = Hyperparams(embed_size=250, hidden_size=250, num_layers=1,
                          loss_func=nn.CrossEntropyLoss,
                          learning_rate=0.03, optimizer=optim.Adam, batch_size=250)

dataloader, model, optimizer, criterion = initialize_data_model_optim_loss(hyperparams)

train(100, dataloader, model, criterion, optimizer)


  9%|███████▍                                                                        | 6/65 [30:04<4:49:23, 294.30s/it]

In [ ]:
for _ in range(10):
    generate_example(model)